In [40]:
import cPickle as pickle
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
import numpy as np
import sys
import os.path
sys.path.append(os.path.join('..', 'TFIDF'))
sys.path.append(os.path.join('..', 'util'))
from timer import Timer

In [41]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [42]:
questions = pickle.load(open("lstOfQuestions.pkl", "rb"))
questions = questions[:10000]

words_dict = {}
question_texts = []
total = len(questions)
count = 0
for question in questions :
    if count % (total / 10) == 0:
        print count
    VALID_TAGS = ['p']

    soup = BeautifulSoup(question['body'])

    VALID_TAGS = ['p']
    INVALID_TAGS = ['code', 'a']

    text = ' '.join([s.get_text() for s in soup('p')])

    tokens = nltk.word_tokenize(text)

    stemmer = PorterStemmer()

    words = [stemmer.stem(token.encode('ascii', 'ignore').lower()) for token in tokens]

    question_texts.append(words)
    
    for word in words :
        if word in words_dict :
            words_dict[word] += 1
        else :
            words_dict[word] = 1
    
    count += 1
    
id_hash = {}
count = 0
for question in questions:
    if question['question_id'] not in id_hash:
        id_hash[question['question_id']] = 1
        count += 1

print count

# Go through dictionary, removing entries that have a value less
# than delta
delta = 2
words_dict2 = {key: value for key, value in words_dict.items()
                 if value >= delta }

word_indices = {}
index = 0
for word in words_dict2 :
    word_indices[word] = index
    index += 1

word_indices = list(word_indices.iteritems())

output = open('wordIndices_md.pkl', 'wb')
pickle.dump(word_indices, output, -1)
output.close()

output = open('questionTexts_md.pkl', 'wb')
pickle.dump(question_texts, output, -1)
output.close()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [43]:
try:
     tfidf = reload(tfidf)
except NameError:
    import TFIDF_numpy as tfidf

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))
    
tfidf.init_globals()
tfidf.load_questions(question_texts)
tfidf.load_indices(word_indices)
tfidf.init_tfs()
serial_tfs = tfidf.create_tfs()
serial_idf = tfidf.calculate_idf()
serial_tfidfs = tfidf.calculate_tfidfs()
# tfidf.calculate_tfidf_norms()
# tfidf.calculate_cossim(question_texts[0])
serial_simhashes = tfidf.calculate_simhashes()

Time for initialization: 0.141933917999
Time for load_questions: 2.14576721191e-06
Time for load_indices: 0.0151240825653
Time for init_tfs: 0.958856105804
Time for create_tfs: 1.36310005188
Time for create_idf: 1.64433193207
Time for calculate_tfidfs: 0.694976091385
Time for calculate_simhashes: 90.0556540489


In [44]:
try :
    tfidf_c = reload(tfidf_c)
except :
    import TFIDF_cython as tfidf_c

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))

# Preprocess for cython code
tfidf_c.init_globals(1, False, "coarse", 64)
tfidf_c.load_questions(question_texts)
tfidf_c.load_indices(word_indices)
tfidf_c.init_tfs()
cython_tfs = np.asarray(tfidf_c.create_tfs())[:,0:len(word_indices)]
cython_idf = np.asarray(tfidf_c.calculate_idf(len(word_indices)))[0:len(word_indices)]
tfidf_c.init_tfidfs()
cython_tfidfs = np.asarray(tfidf_c.calculate_tfidfs())[:,0:len(word_indices)]
cython_simhashes = np.asarray(tfidf_c.calculate_simhashes())
cython_distances = tfidf_c.calculate_distances()

Compiling with /usr/local/bin/clang-omp
Time for Initialization: 1.90734863281e-05
Time for load_questions: 0.073422908783
Time for load_indices: 0.0291738510132
Time for init_tfs: 1.19700288773
Time for create_tfs: 0.266258001328
Time for calculate_idf: 0.222138166428
Time for int_tfidfs: 0.854601860046
Time for calculate_tfidfs: 0.160649061203
Time for calculate_simhashes: 0.420385122299
Time for calculate distances: 1.27499699593


In [45]:
# Verification
print np.linalg.norm(cython_tfs - serial_tfs)
print np.linalg.norm(cython_idf - serial_idf)
print np.linalg.norm(cython_tfidfs - serial_tfidfs)

0.0
3.05905382673e-05
0.000547179457161


In [46]:
#The following are from https://yesteapea.wordpress.com/2013/03/03/counting-the-number-of-set-bits-in-an-integer/
def numBits64(i):
    i = i - ((i >> np.uint64(1)) & np.uint64(0x5555555555555555))
    i = (i & np.uint64(0x3333333333333333)) + ((i >> np.uint64(2)) & np.uint64(0x3333333333333333))
    i = ((i + (i >> np.uint64(4))) & np.uint64(0x0F0F0F0F0F0F0F0F))
    return (i*(np.uint64(0x0101010101010101)))>>np.uint64(56)

def numBits32(i):
    i = i - ((i >> 1) & 0x55555555)
    i = (i & 0x33333333) + ((i >> 2) & 0x33333333)
    i = ((i + (i >> 4)) & 0x0F0F0F0F)
    return (i*(0x01010101))>>24

In [56]:
A = np.array([cython_simhashes] * len(cython_simhashes))
B = np.copy(A)
distances = numBits64(A ^ B.T)

In [57]:
from sklearn.cluster import DBSCAN

In [63]:
db = DBSCAN(metric="precomputed", eps=2, min_samples=3).fit(cython_distances)

In [59]:
sum(map(lambda x: 1 if x >= 0 else 0, db.labels_))

9965

In [60]:
set(db.labels_)

{-1,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30}

In [61]:
all_one_labels = []
counter = 0
for i in xrange(len(db.labels_)):
    if db.labels_[i] == -1:
        counter += 1
    elif db.labels_[i] == 4:
        all_one_labels.append(i)
print counter
print all_one_labels

35
[41, 42, 43, 44]


In [62]:
for i in all_one_labels: 
    print " ".join(question_texts[i])
    print ""

select c1.cert_announc from employe c1 , teacher c1 , teacher_cert_int c1 , certif c1 order by emp_lnam , emp_fnam ; write a queri and save it as a view to creat announc text for all teacher in the databas that have obtain certif . the attribut you creat should be name as below and the string you creat must look exactli as in the solut for full credit . order by last name then first name . it is suppos to be name cert_announc but thi is not an attribut in the er diagram . do n't know how to creat it exactli becaus when i run thi it say invalid identifi . ani help is appreci .

i am load anoth chart upon mous click from an exist d3 chart . the chart load , but then the tooltip fail i.e . seem to be disabl . load onli the specif div doe not give me a chart at all . is there ani possibl way to see the tooltip on the second chart ? file1 : home.html file2 : kp_asian.html

am do multipl file upload in the control but the file doe n't get upload i have tri var_dump ( $ imag [ 'name ' ] and e